## Integrantes

- Elsy Camila Silva Velázquez
- Luis Octavio Sánchez Hatadi 
- Guillermo Gerardo Andrés Urbano


3. Menciona lo que son las llamadas al sistema, el estándar POSIX y su uso para implementar concurrencia
en linux-C.

### Programación concurrente en C

In [42]:
%%writefile main.c

#include <stdio.h>
#include <sys/types.h>
#include <unistd.h>

int main() {
    int pid;
    // Creacion de un proceso
    pid = fork();
    printf("ID proceso: %d\n", pid);
    if (pid)
         printf("Hola soy el proceso padre!!\n");
    else        
        printf("Hola soy el proceso hijo!\n");
    return 0;
} 

Writing main.c


In [43]:
%%script bash
gcc main.c -o main.out

./main.out

ID proceso: 24955
Hola soy el proceso padre!!
ID proceso: 0
Hola soy el proceso hijo!


4. Implementa en Python tres versiones de un programa que calcula el factorial de un número entero proporcionado desde el teclado:
- a) Implementa la versión secuencial.
- b) Implementa la versión en la cual se crean dos procesos, qcada uno un valor entero desde el teclado y calcula el factorial correspondiente.
- c) Implementa la versión en la que se crean cinco hilos q

### Factorial de un numero

In [ ]:
def factorial(n):
    product = 1
    for i in range(1, n+1):
        product *= i
    print(f'{product}\n')

## Vesión secuencial

In [39]:
n = int(input())
print("-----")
factorial(n)

5
-----
120



## Versión creación de procesos

In [40]:
import multiprocessing as mp

n1, n2 = int(input()), int(input())
print("-----")

# Creacion de 2 procesos
p1 = mp.Process(target=factorial, args=(n1,))
p2 = mp.Process(target=factorial, args=(n2,))

# Iniciando los procesos
p1.start()
p2.start()

# Haciendo que el padre espere a los dos procesos
p2.join()
p2.join()


5
7
-----
120
5040




## Version creación de hilos

In [37]:
import threading as th

nums = []
threads = []
n = 5 

for i in range(n):
    nums.append(int(input()))

print("-----")
# Creacion de hilos e iniciando el proceso
for i in range(n):
    thread = th.Thread(target=factorial, args=(nums[i],))
    threads.append(thread)
    threads[i].start()

# Espera el padre a los hilos
for i in range(n):
    threads[i].join()

5
6
7
2
4
-----
120

720

5040

2

24



5. Describe la acción de bloqueo, la razón por la cual se utiliza y los métodos que ofrece python para implementarla.

Existen recursos los cuales no se deben acceder al mismo tiempo por dos procesos o más; por lo que es necesario proteger o bloquear el acceso a estos recursos compartidos: memoria, archivos, bases de datos.

La razon por la cual surgen es por la **condición de carrera** (race condition), ocurre cuando dos o más procesos acceden un recurso compartido sin control, de manera que el resultado combinado de este acceso depende del orden de llegada.

El acceso a un recurso compartido por mas de dos tareas, existe la posibilidad de corromper la información.Cuando existen dos procesos pueden modificar la variables, por lo tanto debe de ser sincronizadas. 

Sincronizar es tener cierto orden en los recursos, es un mecanismo para compartir recursos de manera ordenada.


Cuando una tarea solita el acceso para modificarla, pongo un bloqueo, para que el proceso no lea un valor desactualizado


Python ofrece la clase `Lock` para la condición de carrera, ofrecera dos estados `locked` o `unlocked`, de esta manera poder sincronizar los procesos, esta clase continen los metodos:


- **lock.acquire()**: Cuando el estado es _unlocked_, `acquire()` cambia el estado a _locked_. Cuando el estado está _locked_, `acquire()` bloquea hasta que una llamada a `release()` en otro hilo lo cambie a _unlocked_, luego la llamada de `acquire()` lo restablece a bloqueado y regresa.

- **lock.release()**: El metodo `release()`,se utiliza para liberar un bloqueo adquirido, deberia ser solo llamado cuando esta en estado _locked_ ,si no lanzará una exepción _a RuntimeError_.

Ejemplo:

In [41]:
import time
import multiprocessing

def deposit(balance, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value = balance.value + 1
        lock.release()
        
def withdraw(balance, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value = balance.value - 1
        lock.release()
    
        
if __name__ == '__main__':
    balance = multiprocessing.Value('i', 200)
    lock = multiprocessing.Lock()
    d = multiprocessing.Process(target=deposit, args=(balance, lock))
    w = multiprocessing.Process(target=withdraw, args=(balance, lock))
    d.start()
    w.start()
    d.join()
    w.join()
    print(balance.value)

200
